In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import GRU, Activation, Dense, Dropout, Input, Embedding,Flatten
from keras import Model
from keras.layers import Bidirectional
from keras.callbacks import EarlyStopping
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.backend import clear_session
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,TfidfTransformer, HashingVectorizer
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SVMSMOTE
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from numpy import array
from keras.models import Sequential
from keras import layers
from sklearn.model_selection import KFold
from keras.backend import clear_session

clear_session()
# dataset
df=pd.read_csv('/Users/joe/Desktop/language-models-sprint1/data/train.csv')


#Split the data into train and test sets
X = df['text']
y = df['target']


#Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


# apply tokonizer
tokenizer = Tokenizer(lower=True,num_words=40000)
tokenizer.fit_on_texts(X_train)
X_train_tok = tokenizer.texts_to_sequences(X_train)


vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

X_test = tokenizer.texts_to_sequences(X_test)

# padding zerros to insure the sequence of words in a text is equal
# the tweet is 280 long, so I set the padding to 
padding_length=280
X_train_tok=pad_sequences(X_train_tok,padding='post',maxlen=padding_length)
X_test=pad_sequences(X_test,padding='post',maxlen=padding_length)


#early stopping 
early_stop=EarlyStopping(monitor='loss', patience=1, mode='min', verbose=1)


#building model method

#building model method
def build_model(learning_rate,vocab_size,padding_length,
                neurons_of_GRU_layer,neurons_of_second_layer,GRU_activation,second_layer_activation,
                output_layer_activation
                ,optimizer,recurrent_dropout,dropout,embedding_dim
                ):
    inputs= Input(name='inputs',shape=[padding_length])
    layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=padding_length)(inputs)
    layer = Bidirectional(GRU(neurons_of_GRU_layer, activation=GRU_activation,recurrent_dropout=recurrent_dropout))(layer)
    layer = Dense(85, activation=second_layer_activation)(layer)
    layer = Dropout(dropout)(layer)
    layer = Flatten()(layer)
    layer = Dense(1, activation='sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])
    
    return model

# the parameter grid
tuning_parameters = dict(epochs=[20,40,60],
                  batch_size=[200,400,800],
                  vocab_size=[25077], 
                  embedding_dim=[20,40],
                  learning_rate = [0.0001,0.0003,0.0007],
                  neurons_of_GRU_layer=[20,30],
                  neurons_of_second_layer=[45,85],
                  padding_length=[280],
                  GRU_activation=['sigmoid','relu','tanh'],
                  second_layer_activation=['sigmoid','relu','tanh'],
                  output_layer_activation=['sigmoid','relu'],
                  optimizer=['RMSprop','Adam'],
                  recurrent_dropout=[0.15,0.20,0.25],
                  dropout=[0.05,0.10]
         
               
                )

# Running the RandomizedSearchCV

model = KerasClassifier(build_fn=build_model,verbose=1)
tuning = RandomizedSearchCV(estimator=model, param_distributions=tuning_parameters,
                              cv=10, verbose=1, n_jobs=-1,return_train_score=True)

tuning_result = tuning.fit(X_train_tok, y_train,callbacks=[early_stop],validation_split=0.05)
test_accuracy = tuning.score(X_test, y_test)
training_accuracy=tuning.score(X_train_tok,y_train)

print('best score')
print(tuning_result.best_score_)
print('best parameters')
print(tuning_result.best_params_)
print('best test accuracy')
print(test_accuracy)
print('best Training accuracy')
print(training_accuracy)